# 聊天摘要内存缓冲区
在这个演示中，我们使用新的*ChatSummaryMemoryBuffer*来限制聊天历史记录的令牌长度，并迭代地总结所有不适合在内存缓冲区中的消息。如果您希望限制成本和延迟（假设摘要提示使用和生成的令牌少于包括整个历史记录），这可能会很有用。

原始的*ChatMemoryBuffer*允许您在一定数量的令牌后截断历史记录，这对于限制成本和延迟很有用，但也会从聊天历史记录中删除潜在相关的信息。

更新的*ChatSummaryMemoryBuffer*旨在使其更加灵活，以便用户对保留哪些聊天历史记录具有更多控制。


In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [ ]:
from llama_index.core.memory import ChatSummaryMemoryBuffer
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.llms.openai import OpenAI as OpenAiLlm
import tiktoken

首先，我们模拟一些聊天记录，这些记录无法完全放入内存缓冲区中。


In [ ]:
chat_history = [    ChatMessage(role="user", content="什么是LlamaIndex?"),    ChatMessage(        role="assistant",        content="LlamaIndex是用于构建LLM应用程序的主要数据框架",    ),    ChatMessage(role="user", content="你能给我一些更多的细节吗?"),    ChatMessage(        role="assistant",        content="""LlamaIndex是用于构建上下文增强的LLM应用程序的框架。上下文增强是指在您的私人或领域特定数据之上应用LLM的任何用例。一些常见的用例包括以下内容：        问答聊天机器人（通常称为RAG系统，代表“检索增强生成”），文档理解和提取，能够进行研究并采取行动的自主代理        LlamaIndex提供了从原型到生产构建任何上述用例的工具。这些工具允许您同时摄取/处理这些数据并实现将数据访问与LLM提示相结合的复杂查询工作流。""",    ),]

我们设置了summarizer_llm，并创建了一个*ChatSummaryMemoryBuffer*实例。


In [ ]:
model = "gpt-4-0125-preview"
summarizer_llm = OpenAiLlm(model_name=model, max_tokens=256)
tokenizer_fn = tiktoken.encoding_for_model(model).encode
memory = ChatSummaryMemoryBuffer.from_defaults(
    chat_history=chat_history,
    llm=summarizer_llm,
    token_limit=2,
    tokenizer_fn=tokenizer_fn,
)

history = memory.get()

在打印历史记录时，我们可以观察到较早的消息已经被总结了。


In [ ]:
print(history)

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='The user inquired about LlamaIndex, a leading data framework for developing LLM applications. The assistant explained that LlamaIndex is used for building context-augmented LLM applications, giving examples such as Question-Answering Chatbots, Document Understanding and Extraction, and Autonomous Agents. It was mentioned that LlamaIndex provides tools for ingesting and processing data, as well as implementing complex query workflows combining data access with LLM prompting.', additional_kwargs={})]


让我们添加一些新的聊天记录。


In [ ]:
new_chat_history = [
    ChatMessage(role="user", content="Why context augmentation?"),
    ChatMessage(
        role="assistant",
        content="LLMs offer a natural language interface between humans and data. Widely available models come pre-trained on huge amounts of publicly available data. However, they are not trained on your data, which may be private or specific to the problem you're trying to solve. It's behind APIs, in SQL databases, or trapped in PDFs and slide decks. LlamaIndex provides tooling to enable context augmentation. A popular example is Retrieval-Augmented Generation (RAG) which combines context with LLMs at inference time. Another is finetuning.",
    ),
    ChatMessage(role="user", content="Who is LlamaIndex for?"),
    ChatMessage(
        role="assistant",
        content="LlamaIndex provides tools for beginners, advanced users, and everyone in between. Our high-level API allows beginner users to use LlamaIndex to ingest and query their data in 5 lines of code. For more complex applications, our lower-level APIs allow advanced users to customize and extend any module—data connectors, indices, retrievers, query engines, reranking modules—to fit their needs.",
    ),
]
memory.put(new_chat_history[0])
memory.put(new_chat_history[1])
memory.put(new_chat_history[2])
memory.put(new_chat_history[3])
history = memory.get()

历史记录现在将被更新，包含最新的信息摘要。


In [ ]:
print(history)

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='The user asked about LlamaIndex and why context augmentation is important. The assistant explained that LlamaIndex is for building context-augmented LLM applications, which are necessary because LLMs are not trained on specific private or problem-specific data. LlamaIndex provides tools for beginners and advanced users to ingest and query data easily, as well as customize modules for more complex applications.', additional_kwargs={})]


使用更长的*token_limit_full_text*可以让用户控制保留完整聊天记录和摘要之间的平衡。


In [ ]:
memory = ChatSummaryMemoryBuffer.from_defaults(
    chat_history=chat_history + new_chat_history,
    llm=summarizer_llm,
    token_limit=256,
    tokenizer_fn=tokenizer_fn,
)
print(memory.get())

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='The user inquired about LlamaIndex, and the assistant explained that it is a data framework for creating context-augmented LLM applications. These applications can include question-answering chatbots, document understanding and extraction, and autonomous agents for research and actions. LlamaIndex provides tools for building these applications from prototype to production, enabling data processing, complex query workflows, and LLM prompting.', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='Why context augmentation?', additional_kwargs={}), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="LLMs offer a natural language interface between humans and data. Widely available models come pre-trained on huge amounts of publicly available data. However, they are not trained on your data, which may be private or specific to the problem you're trying to solve. It's behind APIs, in SQL databases,